In [1]:
import syft
import syft.nn as nn
from syft.controller import tensors, models
import imp
imp.reload(syft.controller)
imp.reload(syft.nn)
imp.reload(syft)

import numpy as np
from syft import FloatTensor


/Users/perusha/anaconda/envs/python3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
Import warning: PyTorch capabilities not available due to torch module not found on your system
How to install PyTorch: http://pytorch.org/

In [2]:
# Testing fit - high level: 

import syft.optim as optim

OPTIMIZER_TYPE = 'adam'

model = nn.Sequential([
    nn.Linear(3,4),
    nn.Sigmoid(),
    nn.Linear(4,2),
    nn.Sigmoid()
])

model.summary()

# Define the optimizer you want to test above
if OPTIMIZER_TYPE == 'sgd':
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
elif OPTIMIZER_TYPE == 'rmsprop':
    # **NOTE** RMSProp doesn't converge with the recommended params for this fake dataset.
    optimizer = optim.RMSProp(model.parameters(), lr=0.01, rho=0.95)
elif OPTIMIZER_TYPE == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=0.1)

#criterion = syft.nn.NLLLoss()
criterion = syft.nn.CrossEntropyLoss()

input = FloatTensor([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]],autograd=True)
target = FloatTensor([[0,0],[0,0],[0,1],[0,1]],autograd=True)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_1 (Linear)            (None, 4)                 12
_________________________________________________________________
sigmoid_2 (Sigmoid)          (dynamic)                 0
_________________________________________________________________
linear_3 (Linear)            (None, 2)                 8
_________________________________________________________________
sigmoid_4 (Sigmoid)          (dynamic)                 0
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________



In [4]:
model

<syft.nn.linear at 1>
	 W:[syft.FloatTensor:1 grad:51 size:3x4 c:[15, 48] p:[] init:]
	 A:[syft.FloatTensor:15 grad:47 size:2x4 c:[16] p:[13, 1] init:mm]

<syft.nn.sigmoid at 2>
	 A:[syft.FloatTensor:16 grad:41 size:2x4 c:[17, 40, 42] p:[15] init:sigmoid]

<syft.nn.linear at 3>
	 W:[syft.FloatTensor:2 grad:52 size:4x2 c:[17, 39] p:[] init:]
	 A:[syft.FloatTensor:17 grad:38 size:2x2 c:[18] p:[16, 2] init:mm]

<syft.nn.sigmoid at 4>
	 A:[syft.FloatTensor:18 grad:30 size:2x2 c:[19, 25, 31, 105] p:[17] init:sigmoid]


In [3]:
final_loss = model.fit(input=input,
                       target=target,                       
                       batch_size=2,
                       criterion=criterion,
                       optim=optimizer,
                       iters=2,log_interval=4)
final_loss

Number of Batches:2


100000

In [6]:
tensors[30]

[[ 0.  0.]
 [ 0.  0.]]
[syft.FloatTensor:30 grad:None size:2x2 c:[31] p:[16, 29] init:mul_elem]

	-----------children-----------
	[syft.FloatTensor:31 grad:None size:1 c:[32] p:[30] init:sum_-1_False]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:16 grad:None size:2x2 c:[30, 32] p:[] init:]
	[syft.FloatTensor:29 grad:None size:2x2 c:[30] p:[28] init:log]
	------------------------------



In [ ]:
# Testing a single forward and backward pass: 

model = nn.Sequential([
    nn.Linear(3,4),
    nn.Sigmoid(),
    nn.Linear(4,1),
    nn.Sigmoid()
])

In [3]:
input_data = np.array([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]])
target_data = np.array([[0],[0],[1],[1]])
input = FloatTensor([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]], autograd=True)
target = FloatTensor([[0],[0],[1],[1]], autograd=True)

In [4]:
pred = model(input)
#loss = (pred - target) ** 2
#loss.backward()

In [5]:
pred

[[ 0.5335057]
 [ 0.5318952]
 [ 0.5441882]
 [ 0.5424554]]
[syft.FloatTensor:16 grad:None size:4x1 c:[] p:[15] init:sigmoid]

	-----------creators-----------
	[syft.FloatTensor:15 grad:None size:4x1 c:[16] p:[12, 14] init:add_elem]
	------------------------------



In [6]:
# Testing MSELoss: 
loss = (nn.MSELoss()).forward(pred,target)
print(loss)
print(((pred-target)**2).sum())

  1.140322 

  1.140322 



In [6]:
#Manual Cross Entropy Calc: 
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
print(pred.to_numpy())
print(softmax(pred.to_numpy()))
-1*np.sum(target_data*np.log1p(softmax(pred.to_numpy())))

[[ 0.7043499]
 [ 0.7426865]
 [ 0.7711535]
 [ 0.7993531]]
[[ 0.23765225]
 [ 0.24693993]
 [ 0.25407058]
 [ 0.26133724]]


-0.45856718401967267

In [6]:
# Testing Cross Entropy Loss
loss = (nn.CrossEntropyLoss()).forward(pred,target)
print(loss)

  0. 


Just testing the printing works: 

In [7]:
model

<syft.nn.linear at 1>
	 W:[syft.FloatTensor:1 grad:None size:3x4 c:[7] p:[] init:]
	 W:[syft.FloatTensor:2 grad:None size:1x4 c:[8] p:[] init:]
	 A:[syft.FloatTensor:10 grad:None size:4x4 c:[11] p:[7, 9] init:add_elem]

<syft.nn.sigmoid at 2>
	 A:[syft.FloatTensor:11 grad:None size:4x4 c:[12] p:[10] init:sigmoid]

<syft.nn.linear at 3>
	 W:[syft.FloatTensor:3 grad:None size:4x1 c:[12] p:[] init:]
	 W:[syft.FloatTensor:4 grad:None size:1x1 c:[13] p:[] init:]
	 A:[syft.FloatTensor:15 grad:None size:4x1 c:[16] p:[12, 14] init:add_elem]

<syft.nn.sigmoid at 4>
	 A:[syft.FloatTensor:16 grad:None size:4x1 c:[17, 22] p:[15] init:sigmoid]


In [8]:
pred

[[ 0.5335057]
 [ 0.5318952]
 [ 0.5441882]
 [ 0.5424554]]
[syft.FloatTensor:16 grad:None size:4x1 c:[17, 22] p:[15] init:sigmoid]

	-----------children-----------
	[syft.FloatTensor:17 grad:None size:4x1 c:[18] p:[16] init:copy_autograd_flip]
	[syft.FloatTensor:22 grad:None size:1 c:[] p:[16, 6] init:crossentropyloss]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:15 grad:None size:4x1 c:[16] p:[12, 14] init:add_elem]
	------------------------------



In [9]:
loss

[ 0.]
[syft.FloatTensor:22 grad:None size:1 c:[] p:[16, 6] init:crossentropyloss]

	-----------creators-----------
	[syft.FloatTensor:16 grad:None size:4x1 c:[17, 22] p:[15] init:sigmoid]
	[syft.FloatTensor:6 grad:None size:4x1 c:[20, 22] p:[] init:]
	------------------------------



In [10]:
print((tensors[22]).autograd())
print((tensors[16]).autograd())
print((tensors[6]).autograd())
print((tensors[17]).autograd())


True
True
False
False


In [11]:
tensors[17]

[[ 0.5335057]
 [ 0.5318952]
 [ 0.5441882]
 [ 0.5424554]]
[syft.FloatTensor:17 grad:None size:4x1 c:[18] p:[16] init:copy_autograd_flip]

	-----------children-----------
	[syft.FloatTensor:18 grad:None size:4x1 c:[19] p:[17] init:softmax-1]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:16 grad:None size:4x1 c:[17, 22] p:[15] init:sigmoid]
	------------------------------



In [12]:
loss.backward()

In [19]:
pred

[[ 0.5335057]
 [ 0.5318952]
 [ 0.5441882]
 [ 0.5424554]]
[syft.FloatTensor:16 grad:28 size:4x1 c:[17, 22] p:[15] init:sigmoid]

	-----------children-----------
	[syft.FloatTensor:17 grad:None size:4x1 c:[18, 29, 32] p:[16] init:copy_autograd_flip]
	[syft.FloatTensor:22 grad:23 size:1 c:[] p:[16, 6] init:crossentropyloss]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:15 grad:35 size:4x1 c:[16] p:[12, 14] init:add_elem]
	------------------------------



In [20]:
loss

[ 0.]
[syft.FloatTensor:22 grad:23 size:1 c:[] p:[16, 6] init:crossentropyloss]

	-----------creators-----------
	[syft.FloatTensor:16 grad:28 size:4x1 c:[17, 22] p:[15] init:sigmoid]
	[syft.FloatTensor:6 grad:None size:4x1 c:[20, 22] p:[] init:]
	------------------------------



In [18]:
#tensors[18]
tensors[28]
#tensors[24]
#tensors[27]

[-0.1520445]
[syft.FloatTensor:28 grad:None size:1 c:[] p:[27, 23] init:mul_elem]

	-----------creators-----------
	[syft.FloatTensor:27 grad:None size:1 c:[28] p:[26] init:sum_-1_False]
	[syft.FloatTensor:23 grad:None size:1 c:[28] p:[] init:emptyTensorCopy]
	------------------------------



In [21]:
model

<syft.nn.linear at 1>
	 W:[syft.FloatTensor:1 grad:48 size:3x4 c:[7, 45] p:[] init:]
	 W:[syft.FloatTensor:2 grad:52 size:1x4 c:[8] p:[] init:]
	 A:[syft.FloatTensor:10 grad:44 size:4x4 c:[11] p:[7, 9] init:add_elem]

<syft.nn.sigmoid at 2>
	 A:[syft.FloatTensor:11 grad:38 size:4x4 c:[12, 37, 39] p:[10] init:sigmoid]

<syft.nn.linear at 3>
	 W:[syft.FloatTensor:3 grad:53 size:4x1 c:[12, 36] p:[] init:]
	 W:[syft.FloatTensor:4 grad:57 size:1x1 c:[13] p:[] init:]
	 A:[syft.FloatTensor:15 grad:35 size:4x1 c:[16] p:[12, 14] init:add_elem]

<syft.nn.sigmoid at 4>
	 A:[syft.FloatTensor:16 grad:28 size:4x1 c:[17, 22] p:[15] init:sigmoid]


In [22]:
tensors[12]

[[ 0.1342238]
 [ 0.1277542]
 [ 0.1772151]
 [ 0.1702316]]
[syft.FloatTensor:12 grad:35 size:4x1 c:[15] p:[11, 3] init:mm]

	-----------children-----------
	[syft.FloatTensor:15 grad:35 size:4x1 c:[16] p:[12, 14] init:add_elem]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:11 grad:38 size:4x4 c:[12, 37, 39] p:[10] init:sigmoid]
	[syft.FloatTensor:3 grad:53 size:4x1 c:[12, 36] p:[] init:]
	------------------------------



In [23]:
tensors[11]

[[ 0.6186889  0.6398186  0.3602833  0.3954641]
 [ 0.5670761  0.6698359  0.4540931  0.4075304]
 [ 0.5846717  0.7317994  0.3225332  0.4490698]
 [ 0.5319379  0.7570617  0.4128572  0.4615231]]
[syft.FloatTensor:11 grad:38 size:4x4 c:[12, 37, 39] p:[10] init:sigmoid]

	-----------children-----------
	[syft.FloatTensor:12 grad:35 size:4x1 c:[15] p:[11, 3] init:mm]
	[syft.FloatTensor:37 grad:None size:4x4 c:[53] p:[11] init:transpose]
	[syft.FloatTensor:39 grad:None size:4x4 c:[40, 43] p:[11] init:copy_autograd_flip]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:10 grad:44 size:4x4 c:[11] p:[7, 9] init:add_elem]
	------------------------------

